# Behavior Cloning Project

The goals / steps of this project are the following:
* Use the simulator to collect data of good driving behavior
* Build, a convolution neural network in Keras that predicts steering angles from images
* Train and validate the model with a training and validation set
* Test that the model successfully drives around track one without leaving the road
* Summarize the results with a written report

The simulator can be found here:

https://github.com/udacity/self-driving-car-sim 
https://github.com/endymioncheung/CarND-MacCatalinaSimulator (macOS)

### Collecting Training Data
In order to start collecting training data, you'll need to do the following:

1. Enter Training Mode in the simulator.
2. Start driving the car to get a feel for the controls.
3. When you are ready, hit the record button in the top right to start recording.
4. Continue driving for a few laps or till you feel like you have enough data.
5. Hit the record button in the top right again to stop recording.

#### Strategies for Collecting Data

    the car should stay in the center of the road as much as possible
    if the car veers off to the side, it should recover back to center
    driving counter-clockwise can help the model generalize
    flipping the images is a quick way to augment the data
    collecting data from the second track can also help generalize the model
    we want to avoid overfitting or underfitting when training the model
    knowing when to stop collecting more data

Data will be saved from the recorder as follows:

1. IMG folder - this folder contains all the frames of your driving.
2. driving_log.csv - each row in this sheet correlates your image with the steering angle, throttle, brake, and speed of your car. You'll mainly be using the steering angle.


## Training The Network

I will use Keras to train a network to do the following:

1. Take in an image from the center camera of the car. This is the input to your neural network.
2. Output a new steering angle for the car.

The following network will be used to verify that everything is working properly and will be a flattened image connected to a single output node. 
This single output node will predict my steering angle, which makes this a regression network and no activation function will be applied. 
This will try to minimize the error that the network predicts and the ground truth steering measurement

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt

lines = []
# Loading in the data from the csv file
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

# Extracting the image and measurements 
images = []
measurements = []
for line in lines[1:]:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = 'data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
X_train = np.array(images)
y_train = np.array(measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense
model = Sequential()
model.add(Flatten(input_shape = (160,320,3)))
model.add(Dense(1))

# Using mean square error for regression network
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle = True, nb_epoch=7)

model.save('model.h5')

NOTE: cv2.imread will get images in BGR format, while drive.py uses RGB. One way to keep the same image formatting is to do "image = ndimage.imread(current_path)" with "from scipy import ndimage" instead.



## Training The Network Continued 

I will use Keras to continue training the network by adding in the following:

### Lambda Layers
In Keras, lambda layers can be used to create arbitrary functions that operate on each image as it passes through the layer.

In this project, a lambda layer is a convenient way to parallelize image normalization. The lambda layer will also ensure that the model will normalize input images when making predictions in drive.py.

### Flipping Images And Steering Measurements
A effective technique for helping with the left turn bias involves flipping images and taking the opposite sign of the steering measurement. 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Lambda

lines = []
# Loading in the data from the csv file
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

# Extracting the image and measurements 
images = []
measurements = []
for line in lines[1:]:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = 'data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
    
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense
model = Sequential()
# set up lambda layer for normalization
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1))

# Using mean square error for regression network
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle = True, nb_epoch=1)

model.save('model.h5')

## Multiple Cameras

In a real car, we’ll have multiple cameras on the vehicle, and we’ll map recovery paths from each camera. For example, if you train the model to associate a given image from the center camera with a left turn, then you could also train the model to associate the corresponding image from the left camera with a somewhat softer left turn. And you could train the model to associate the corresponding image from the right camera with an even harder left turn.

In that way, you can simulate your vehicle being in different positions, somewhat further off the center line. To read more about this approach, see this paper by NVIDIA: http://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf

Explanation of How Multiple Cameras Work

From the perspective of the left camera, the steering angle would be less than the steering angle from the center camera. From the right camera's perspective, the steering angle would be larger than the angle from the center camera. The next section will discuss how this can be implemented in your project although there is no requirement to use the left and right camera images.

Multiple Cameras in This Project

For this project, recording recoveries from the sides of the road back to center is effective. But it is also possible to use all three camera images to train the model. When recording, the simulator will simultaneously save an image for the left, center and right cameras. Each row of the csv log file, driving_log.csv, contains the file path for each camera as well as information about the steering measurement, throttle, brake and speed of the vehicle.

During training, I want to feed the left and right camera images to your model as if they were coming from the center camera. This way, you can teach your model how to steer if the car drifts off to the left or the right.

Figuring out how much to add or subtract from the center angle will involve some experimentation.

During prediction (i.e. "autonomous mode"), you only need to predict with the center camera image.

It is not necessary to use the left and right images to derive a successful model. Recording recovery driving from the sides of the road is also effective.


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Lambda

lines = []
# Loading in the data from the csv file
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

# create adjusted steering measurements for the side camera images
correction = 0.2 # this is a parameter to tune

# Extracting the image and measurements 
images = []
measurements = []
for line in lines[1:]:
    for i in range(3):
        source_path = line[i]
        filename = source_path.split('/')[-1]
        current_path = 'data/IMG/' + filename
        image = cv2.imread(current_path)
        images.append(image)
        if i == 0: # Center
            measurement = float(line[3])
        elif i == 1: # Left
            measurement = float(line[3]) + correction
        elif i == 2: # Right
            measurement = float(line[3]) - correction        
        measurements.append(measurement)
    
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
    
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense
model = Sequential()
# set up lambda layer for normalization
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1))

# Using mean square error for regression network
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle = True, nb_epoch=1)

model.save('model.h5')

## Cropping2D Layer

Keras provides the Cropping2D layer for image cropping within the model. This is relatively fast, because the model is parallelized on the GPU, so many images are cropped simultaneously.

By contrast, image cropping outside the model on the CPU is relatively slow.

Also, by adding the cropping layer, the model will automatically crop the input images when making predictions in drive.py.

The Cropping2D layer might be useful for choosing an area of interest that excludes the sky and/or the hood of the car.

Cropping Layer Code Example
    
    from keras.models import Sequential, Model
    from keras.layers import Cropping2D
    import cv2

    # set up cropping2D layer
    model = Sequential()
    model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))


The example above crops:

* 50 rows pixels from the top of the image
* 20 rows pixels from the bottom of the image
* 0 columns of pixels from the left of the image
* 0 columns of pixels from the right of the image

## Network Architecture Updates

I am going to implement the NVIDIA Self Driving Car network published here: https://devblogs.nvidia.com/deep-learning-self-driving-cars/

The first layer of the network performs image normalization. The normalizer is hard-coded and is not adjusted in the learning process. Performing normalization in the network allows the normalization scheme to be altered with the network architecture, and to be accelerated via GPU processing.

The convolutional layers are designed to perform feature extraction, and are chosen empirically through a series of experiments that vary layer configurations. We then use strided convolutions in the first three convolutional layers with a 2×2 stride and a 5×5 kernel, and a non-strided convolution with a 3×3 kernel size in the final two convolutional layers.

We follow the five convolutional layers with three fully connected layers, leading to a final output control value which is the inverse-turning-radius. The fully connected layers are designed to function as a controller for steering, but we noted that by training the system end-to-end, it is not possible to make a clean break between which parts of the network function primarily as feature extractor, and which serve as controller.




In [1]:
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Lambda, Cropping2D
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Conv2D

lines = []
# Loading in the data from the csv file
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

# create adjusted steering measurements for the side camera images
correction = 0.2 # this is a parameter to tune

# Extracting the image and measurements 
images = []
measurements = []
for line in lines[1:]:
    for i in range(3):
        source_path = line[i]
        filename = source_path.split('/')[-1]
        current_path = 'data/IMG/' + filename
        imageBGR = cv2.imread(current_path)
        # Images in drive.py are read in as RGB
        image = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2RGB)
        images.append(image)
        if i == 0: # Center
            measurement = float(line[3])
        elif i == 1: # Left
            measurement = float(line[3]) + correction
        elif i == 2: # Right
            measurement = float(line[3]) - correction        
        measurements.append(measurement)

# Augmenting data to avoid left bias on the track
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
    
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

model = Sequential()
# set up lambda layer for normalization
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
# cropping 70 pixels from top of image (trees) and 25 pixels from bottom of image (hood of car)
model.add(Cropping2D(cropping=((70,25),(0,0))))

# NVIDIA architecture and including a dropout layer for redundancy
model.add(Conv2D(24,5,5, subsample = (2,2), activation = "relu"))
model.add(Conv2D(36,5,5, subsample = (2,2), activation = "relu"))
model.add(Conv2D(48,5,5, subsample = (2,2), activation = "relu"))
model.add(Conv2D(64,3,3, activation = "relu"))
model.add(Conv2D(64,3,3, activation = "relu"))
model.add(Flatten())
model.add(Dropout(0.5))   
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

# Using mean square error for regression network
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle = True, nb_epoch=5)

# Saving to file for use in drive.py
model.save('model.h5')

Using TensorFlow backend.
/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/carnd-ter

Instructions for updating:
Colocations handled automatically by placer.


/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/ipykernel_launcher.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/ipykernel_launcher.py:59: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/ipykernel_launcher.py:60: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/ipykernel_launcher.py:61: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/ipykernel_launcher.py:62: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/opt/anaconda3/envs/carnd-t

Instructions for updating:
Use tf.cast instead.
Train on 38572 samples, validate on 9644 samples
Epoch 1/5
38572/38572 [==============================] - 143s 4ms/step - loss: 0.0188 - val_loss: 0.0184
Epoch 2/5
38572/38572 [==============================] - 147s 4ms/step - loss: 0.0156 - val_loss: 0.0184
Epoch 3/5
38572/38572 [==============================] - 146s 4ms/step - loss: 0.0145 - val_loss: 0.0174
Epoch 4/5
38572/38572 [==============================] - 145s 4ms/step - loss: 0.0138 - val_loss: 0.0204
Epoch 5/5
38572/38572 [==============================] - 150s 4ms/step - loss: 0.0130 - val_loss: 0.0178


## Visualizin Loss

Outputting Training and Validation Loss Metrics
In Keras, the model.fit() and model.fit_generator() methods have a verbose parameter that tells Keras to output loss metrics as the model trains. The verbose parameter can optionally be set to verbose = 1 or verbose = 2.

Setting model.fit(verbose = 1) will

output a progress bar in the terminal as the model trains.
output the loss metric on the training set as the model trains.
output the loss on the training and validation sets after each epoch.
With model.fit(verbose = 2), Keras will only output the loss on the training set and validation set after each epoch.



In [ ]:
from keras.models import Model
import matplotlib.pyplot as plt

history_object = model.fit_generator(train_generator, samples_per_epoch =
    len(train_samples), validation_data = 
    validation_generator,
    nb_val_samples = len(validation_samples), 
    nb_epoch=3, verbose=1)

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

# Save the model
model.save('model.h5')

## Generators 

The images captured in the car simulator are much larger than the images encountered in the Traffic Sign Classifier Project, a size of 160 x 320 x 3 compared to 32 x 32 x 3. Storing 10,000 traffic sign images would take about 30 MB but storing 10,000 simulator images would take over 1.5 GB. That's a lot of memory! Not to mention that preprocessing data can change data types from an int to a float, which can increase the size of the data by a factor of 4.

Generators can be a great way to work with large amounts of data. Instead of storing the preprocessed data in memory all at once, using a generator you can pull pieces of the data and process them on the fly only when you need them, which is much more memory-efficient.

A generator is like a coroutine, a process that can run separately from another main routine, which makes it a useful Python function. Instead of using return, the generator uses yield, which still returns the desired output values but saves the current values of all the generator's variables. When the generator is called a second time it re-starts right after the yield statement, with all its variables set to the same values as before.



In [ ]:
import os
import csv
import warnings
warnings.filterwarnings("ignore")
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Lambda, Cropping2D
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Conv2D
import sklearn
from sklearn.utils import shuffle
import matplotlib.image as mpimg

samples = []
# Loading in the data from the csv file
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

# create adjusted steering measurements for the side camera images
correction = 0.2 # this is a parameter to tune

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            measurements = []
            for batch_sample in batch_samples:
                for i in range(3):
                    try:
                        source_path = batch_sample[i]
                        filename = source_path.split('/')[-1]
                        current_path = 'data/IMG/' + filename
                        image = mpimg.imread(current_path)
                        images.append(image)

                        # Augmenting the images to avoid a bias on the left side
                        images.append(cv2.flip(image,1))

                        if i == 0: # Center
                            measurement = float(line[3])
                        elif i == 1: # Left
                            measurement = float(line[3]) + correction
                        elif i == 2: # Right
                            measurement = float(line[3]) - correction        
                        measurements.append(measurement)

                        # Augmenting the images to avoid a bias on the left side
                        measurements.append(measurement*-1.0)
                    except:
                        continue

            X_train = np.array(images)
            y_train = np.array(measurements)
            yield sklearn.utils.shuffle(X_train, y_train)

# Set our batch size
batch_size=32

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

model = Sequential()
# set up lambda layer for normalization
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
# cropping 70 pixels from top of image (trees) and 25 pixels from bottom of image (hood of car)
model.add(Cropping2D(cropping=((70,25),(0,0))))

# NVIDIA architecture and including a dropout layer for redundancy
model.add(Conv2D(24,5,5, subsample = (2,2), activation = "relu"))
model.add(Conv2D(36,5,5, subsample = (2,2), activation = "relu"))
model.add(Conv2D(48,5,5, subsample = (2,2), activation = "relu"))
model.add(Conv2D(64,3,3, activation = "relu"))
model.add(Conv2D(64,3,3, activation = "relu"))
model.add(Flatten())
model.add(Dropout(0.5))   
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
history_object = model.fit_generator(train_generator, steps_per_epoch=np.ceil(len(train_samples)/batch_size), 
                    validation_data=validation_generator, 
                    validation_steps=np.ceil(len(validation_samples)/batch_size), 
                    epochs=5, verbose=1)

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

# Save the model
model.save('model.h5')

## Validating The Network

In order to validate your network, you'll want to compare model performance on the training set and a validation set. The validation set should contain image and steering data that was not used for training. A rule of thumb could be to use 80% of your data for training and 20% for validation or 70% and 30%. Be sure to randomly shuffle the data before splitting into training and validation sets.

If model predictions are poor on both the training and validation set (for example, mean squared error is high on both), then this is evidence of underfitting. Possible solutions could be to

    increase the number of epochs
    add more convolutions to the network.
    
When the model predicts well on the training set but poorly on the validation set (for example, low mean squared error for training set, high mean squared error for validation set), this is evidence of overfitting. If the model is overfitting, a few ideas could be to

    use dropout or pooling layers
    use fewer convolution or fewer fully connected layers
    collect more data or further augment the data set
    
Ideally, the model will make good predictions on both the training and validation sets. The implication is that when the network sees an image, it can successfully predict what angle was being driven at that moment.

## Testing The Network

Once you're satisfied that the model is making good predictions on the training and validation sets, you can test your model by launching the simulator and entering autonomous mode.

The car will just sit there until your Python server connects to it and provides it steering angles. Here’s how you start your Python server:

    python drive.py model.h5

Once the model is up and running in drive.py, you should see the car move around (and hopefully not off) the track! If your model has low mean squared error on the training and validation sets but is driving off the track, this could be because of the data collection process. It's important to feed the network examples of good driving behavior so that the vehicle stays in the center and recovers when getting too close to the sides of the road.

In [2]:
!python drive.py model.h5 run2

Using TensorFlow backend.
/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/carnd-ter

0.038860708475112915 6.587560999999998
0.037107814103364944 6.581079999999997
0.02669389173388481 6.574724799999998
0.046359818428754807 6.567709999999997
0.046359818428754807 6.580175199999997
0.060193635523319244 6.573025799999997
0.030528753995895386 6.545785399999997
0.03276049345731735 6.537500799999997
0.03276049345731735 6.548426199999997
0.05307669937610626 6.539869599999997
0.049276772886514664 6.530900399999997
0.03259618580341339 6.5215995999999965
0.03259618580341339 6.531378799999996
0.053260982036590576 6.521747399999996
0.05758322775363922 6.492447199999996
0.05758322775363922 6.5010869999999965
0.08737519383430481 6.490418199999996
0.09556783735752106 6.479431999999996
0.09556783735752106 6.487315799999996
0.10824638605117798 6.476003199999996
0.09712262451648712 6.464405999999997
0.11464762687683105 6.452525999999996
0.11464762687683105 6.459285999999997
0.07733361423015594 6.428163199999997
0.09033012390136719 6.415238999999997
0.09033012390136719 6.420884799999997
0.

0.018685651943087578 6.096291999999995
-0.024135742336511612 6.095924799999994
0.06044436991214752 6.095547599999994
0.06044436991214752 6.095170399999994
0.006291215308010578 6.094813599999994
-0.03151790425181389 6.094446999999994
-0.01021597720682621 6.094151999999995
-0.01021597720682621 6.093776999999995
0.029644887894392014 6.093269399999994
-0.014287017285823822 6.092942799999995
0.0020429783035069704 6.092545799999995
0.0020429783035069704 6.092168799999994
0.025035832077264786 6.091812199999994
0.03949533402919769 6.091404999999995
0.09218569844961166 6.091037999999994
0.09218569844961166 6.090660999999994
0.04897446185350418 6.090294199999994
0.045909084379673004 6.089947999999994
0.045909084379673004 6.0895717999999945
0.042582087218761444 6.089348599999995
0.07644209265708923 6.088883599999995
0.04712819308042526 6.088437199999995
0.04712819308042526 6.088060799999995
0.056442972272634506 6.087663999999995
0.034715309739112854 6.087358599999995
0.08217728137969971 6.0869423

0.020199928432703018 6.016254399999989
0.03645205870270729 6.015897599999989
0.03645205870270729 6.0155207999999885
0.026935307309031487 6.0152255999999875
0.01290945801883936 6.014799399999989
0.053897738456726074 6.014412999999989
0.0513262003660202 6.0140467999999885
0.0513262003660202 6.013670599999989
0.03565341234207153 6.013253599999988
0.017802484333515167 6.012927599999989
0.026041783392429352 6.012561799999989
0.026041783392429352 6.012185999999989
0.014871448278427124 6.011769399999989
0.057204924523830414 6.011372399999988
0.08185311406850815 6.010995399999988
0.0751766487956047 6.010618399999988
0.0751766487956047 6.0102413999999875
0.07343252003192902 6.009905199999988
0.07671228796243668 6.0096003999999885
0.07671228796243668 6.009225599999988
0.03352492302656174 6.0088405999999885
0.0796484425663948 6.008434999999989
0.032740890979766846 6.008059399999989
0.032740890979766846 6.007683799999988
0.04400554299354553 6.007226599999988
0.050335485488176346 6.006920799999988


0.015021483413875103 5.936869199999984
-0.03707263246178627 5.936512199999983
-0.011266750283539295 5.936135199999982
-0.03041132166981697 5.935768399999983
-0.03041132166981697 5.935391599999983
-0.023326419293880463 5.935024999999983
-0.029629793018102646 5.934617799999983
-0.029629793018102646 5.934240599999983
-0.02563449740409851 5.933883799999983
-0.013393079861998558 5.933486599999983
-0.013393079861998558 5.933109399999983
-0.04460185393691063 5.932732199999983
-0.05349890515208244 5.932354999999983
-0.03100743144750595 5.931977799999984
-0.03100743144750595 5.931600599999984
-0.08418229967355728 5.931253999999984
-0.040196746587753296 5.930887599999984
-0.08374293893575668 5.930490799999984
-0.08374293893575668 5.930113999999985
-0.05502593517303467 5.929828999999985
-0.032472021877765656 5.929362199999985
-0.032472021877765656 5.928985399999984
-0.04409734532237053 5.928669799999985
-0.04089784994721413 5.928263599999984
0.023555006831884384 5.927826199999984
-0.0026134254876

...

-0.02977345511317253 3.475923599999968
-0.05561666563153267 3.4643675999999677
-0.03823765367269516 3.4677765999999672
-0.019886840134859085 3.4674457999999677
-0.019886840134859085 3.4671049999999677
0.01112452708184719 3.466386799999968
-0.01926809921860695 3.466038599999968
-0.019296973943710327 3.4648131999999676
0.01348472386598587 3.463417599999968
-0.04928705096244812 3.462277199999968
0.0032861672807484865 3.461631999999968
0.0032861672807484865 3.4612267999999675
-0.03328832983970642 3.4610765999999673
-0.026321766898036003 3.4606763999999672
-0.02632206305861473 3.4607453999999676
-0.004963847808539867 3.4607113999999672
-0.026688845828175545 3.4605823999999674
-0.012964727357029915 3.4603359999999674
-0.012964727357029915 3.4599595999999675
0.03893529623746872 3.4596545999999675
-0.027712905779480934 3.4592795999999675
-0.027768801897764206 3.458945399999967
-0.012893131002783775 3.458581399999967
-0.029289428144693375 3.4582481999999675
-0.00525149330496788 3.45785459999996

0.026680316776037216 3.388253199999958
0.04070214182138443 3.3878779999999584
0.04070214182138443 3.387421199999958
0.04070214182138443 3.387044399999958
0.007718891836702824 3.386708399999958
-0.026589207351207733 3.386332399999958
-0.026589207351207733 3.3859767999999577
-0.0035537302028387785 3.3856011999999582
-0.0033748734276741743 3.385194999999958
0.02178931050002575 3.384828999999958
0.02178931050002575 3.384452999999958
-0.019103679805994034 3.3840769999999574
0.037084776908159256 3.3836805999999573
-0.006724337115883827 3.3833041999999574
-0.006724337115883827 3.382937999999957
0.05036924406886101 3.382561799999957
0.0398404635488987 3.3822161999999567
0.0398404635488987 3.3818405999999572
0.07539109140634537 3.381434399999957
0.016804248094558716 3.381098999999957
0.016804248094558716 3.380723599999957
-0.005429316312074661 3.380348199999957
-0.006325997412204742 3.380033999999957
-0.006325997412204742 3.3796597999999567
-0.015427637845277786 3.3791631999999563
-0.0031343766

0.054482221603393555 3.310438399999951
0.054482221603393555 3.3100627999999515
0.017245806753635406 3.3097177999999516
-0.010097801685333252 3.3094345999999515
-0.010097801685333252 3.3090613999999516
0.049224525690078735 3.308616799999951
0.013937247917056084 3.3081503999999513
0.013937247917056084 3.3077739999999514
0.011728658340871334 3.307407799999951
0.017856206744909286 3.307062199999951
0.017856206744909286 3.306686599999951
-0.029790036380290985 3.306280399999951
0.028523428365588188 3.305893999999951
0.023755209520459175 3.305517599999951
0.017809240147471428 3.3051411999999507
0.03790587559342384 3.304795399999951
0.018974795937538147 3.304419599999951
0.018974795937538147 3.3040437999999512
0.018974795937538147 3.303667999999951
-6.47024717181921e-05 3.3032921999999516
-0.01666056551039219 3.302885799999952
-0.01666056551039219 3.3025093999999515
0.013222992420196533 3.3021329999999516
-0.023663420230150223 3.3017565999999516
-0.023663420230150223 3.3013903999999514
-0.0080

-0.16244512796401978 3.232782199999949
-0.16330857574939728 3.232302399999949
-0.16200374066829681 3.2317285999999488
-0.14422331750392914 3.2313139999999487
-0.16536948084831238 3.2309291999999488
-0.16536948084831238 3.230554399999949
-0.1819247156381607 3.2301693999999492
-0.18764586746692657 3.229722999999949
-0.17280255258083344 3.229407799999949
-0.21043801307678223 3.229042799999949
-0.21043801307678223 3.228667799999949
-0.20028351247310638 3.228384599999949
-0.17229416966438293 3.2279501999999485
-0.17273804545402527 3.227728799999949
-0.15259025990962982 3.227398199999948
-0.15917761623859406 3.2269255999999484
-0.15917761623859406 3.2265529999999485
-0.14238221943378448 3.226037599999948
-0.11745523661375046 3.2255397999999484
-0.12634550034999847 3.225202799999948
-0.12634550034999847 3.224825799999948
-0.16525863111019135 3.224407999999948
-0.14819911122322083 3.223999599999948
-0.14819911122322083 3.223621199999948
-0.12672697007656097 3.223263199999948
-0.108757153153419

-0.21650874614715576 3.154983799999946
-0.21650874614715576 3.1546285999999464
-0.20240195095539093 3.153640999999946
-0.31091901659965515 3.1530387999999467
-0.31091901659965515 3.1526665999999466
-0.32133397459983826 3.152202599999947
-0.34260910749435425 3.151573599999947
-0.36461398005485535 3.152306399999947
-0.36461398005485535 3.1519491999999474
-0.2988893389701843 3.151734799999947
-0.3319856524467468 3.1514721999999464
-0.36154818534851074 3.1513235999999467
-0.36154818534851074 3.150974999999947
-0.2444680631160736 3.1495145999999474
-0.30648839473724365 3.149490999999947
-0.35916319489479065 3.1494027999999474
-0.35916319489479065 3.1490445999999475
-0.34852364659309387 3.147727599999947
-0.3742697834968567 3.1477483999999465
-0.3742697834968567 3.1473791999999468
-0.2977975010871887 3.147509799999947
-0.3679693639278412 3.1470687999999476
-0.3679693639278412 3.1467077999999473
-0.35590338706970215 3.1464793999999467
-0.3900213837623596 3.145121399999947
-0.3900213837623596 

The fourth argument, run1, is the directory in which to save the images seen by the agent. If the directory already exists, it'll be overwritten.

The image file name is a timestamp of when the image was seen.. This information is used by video.py to create a chronological video of the agent driving.

Using video.py

    python video.py run1
    
Creates a video based on images found in the run1 directory. The name of the video will be the name of the directory followed by '.mp4', so, in this case the video will be run1.mp4.

Optionally, one can specify the FPS (frames per second) of the video:

    python video.py run1 --fps 48
    
The video will run at 48 FPS. The default FPS is 60.



In [ ]:
!python video.py run2